In [2]:
from tree_sitter import Language, Parser

In [66]:

Language.build_library(
    # Store the library in the `build` directory
    'my-languages.so',

    # Include one or more languages
    [
        '/home/sasce/PycharmProjects/AutoFL/data/grammars/tree-sitter-java',
        '/home/sasce/PycharmProjects/AutoFL/data/grammars/tree-sitter-python'
    ]
)

False

In [3]:
JAVA_LANGUAGE = Language('/home/sasce/PycharmProjects/AutoFL/notebooks/test/my-languages.so', 'java')

In [4]:
parser = Parser()
parser.set_language(JAVA_LANGUAGE)

In [5]:
code = """
class Lamp {

  // stores the value for light
  // true if light is on
  // false if light is off
  boolean isOn;

  // method to turn on the light
  void turnOn() {
    isOn = true;
    System.out.println("Light on? " + isOn);

  }

  // method to turnoff the light
  void turnOff() {
    isOn = false;
    System.out.println("Light on? " + isOn);
  }
}

class Main {
  public static void main(String[] args) {

    // create objects led and halogen
    Lamp led = new Lamp();
    Lamp halogen = new Lamp();

    // turn on the light by
    // calling method turnOn()
    led.turnOn();

    // turn off the light by
    // calling method turnOff()
    halogen.turnOff();
  }
}
"""

In [6]:
code = bytes(code, "utf8")
tree = parser.parse(code)

In [7]:
tree.root_node.sexp()

'(program (class_declaration name: (identifier) body: (class_body (line_comment) (line_comment) (line_comment) (field_declaration type: (boolean_type) declarator: (variable_declarator name: (identifier))) (line_comment) (method_declaration type: (void_type) name: (identifier) parameters: (formal_parameters) body: (block (expression_statement (assignment_expression left: (identifier) right: (true))) (expression_statement (method_invocation object: (field_access object: (identifier) field: (identifier)) name: (identifier) arguments: (argument_list (binary_expression left: (string_literal (string_fragment)) right: (identifier))))))) (line_comment) (method_declaration type: (void_type) name: (identifier) parameters: (formal_parameters) body: (block (expression_statement (assignment_expression left: (identifier) right: (false))) (expression_statement (method_invocation object: (field_access object: (identifier) field: (identifier)) name: (identifier) arguments: (argument_list (binary_expres

In [8]:
query = JAVA_LANGUAGE.query("""(scoped_identifier scope: (identifier) @identifier.def)""")
captures_a = query.captures(tree.root_node)
print(len(captures_a))

0


In [9]:
query = JAVA_LANGUAGE.query("""
    ((identifier) @identifier)
    ((type_identifier) @type)""")
captures_b = query.captures(tree.root_node)
print(len(captures_b))

28


In [10]:
res = []
for node, _ in captures_b:
    token = code[node.start_byte:node.end_byte]
    res.append(token.decode())

In [11]:
print(res)

['Lamp', 'isOn', 'turnOn', 'isOn', 'System', 'out', 'println', 'isOn', 'turnOff', 'isOn', 'System', 'out', 'println', 'isOn', 'Main', 'main', 'String', 'args', 'Lamp', 'led', 'Lamp', 'Lamp', 'halogen', 'Lamp', 'led', 'turnOn', 'halogen', 'turnOff']


In [12]:
PYTHON_LANGUAGE = Language('/home/sasce/PycharmProjects/AutoFL/notebooks/test/my-languages.so', 'python')
parser = Parser()
parser.set_language(PYTHON_LANGUAGE)

In [13]:
code = """
from collections import Counter
import numpy as np
from multiset import Multiset
from entity.taxonomy import KeywordLabel
from annotation import LFBase


class KeywordLF(LFBase):

    def annotate(self, name: str, content: str) -> np.array:
        node_labels = np.zeros(len(self.taxonomy))
        for _label in self.taxonomy:
            label: KeywordLabel = _label
            intersection = list(label.keywords.intersection(Multiset(content.split())))
            intersection = Counter(intersection)
            node_labels[label.index] = sum([intersection[k] * label.weights[k]
                                            for k in intersection.keys()])

        norm = np.sum(node_labels)
        node_vec = node_labels / norm if norm > 0 else np.zeros(len(self.taxonomy))

        return node_vec

"""

In [14]:
code = bytes(code, "utf8")
tree = parser.parse(code)

In [15]:
tree.root_node.sexp()

'(module (import_from_statement module_name: (dotted_name (identifier)) name: (dotted_name (identifier))) (import_statement name: (aliased_import name: (dotted_name (identifier)) alias: (identifier))) (import_from_statement module_name: (dotted_name (identifier)) name: (dotted_name (identifier))) (import_from_statement module_name: (dotted_name (identifier) (identifier)) name: (dotted_name (identifier))) (import_from_statement module_name: (dotted_name (identifier)) name: (dotted_name (identifier))) (class_definition name: (identifier) superclasses: (argument_list (identifier)) body: (block (function_definition name: (identifier) parameters: (parameters (identifier) (typed_parameter (identifier) type: (type (identifier))) (typed_parameter (identifier) type: (type (identifier)))) return_type: (type (attribute object: (identifier) attribute: (identifier))) body: (block (expression_statement (assignment left: (identifier) right: (call function: (attribute object: (identifier) attribute: (

In [16]:
query = PYTHON_LANGUAGE.query("""
    ((identifier) @identifier)
   """)
captures_b = query.captures(tree.root_node)
print(len(captures_b))

70


In [17]:
res = []
for node, _ in captures_b:
    token = code[node.start_byte:node.end_byte]
    res.append(token.decode())

In [18]:
print(res)

['collections', 'Counter', 'numpy', 'np', 'multiset', 'Multiset', 'entity', 'taxonomy', 'KeywordLabel', 'annotation', 'LFBase', 'KeywordLF', 'LFBase', 'annotate', 'self', 'name', 'str', 'content', 'str', 'np', 'array', 'node_labels', 'np', 'zeros', 'len', 'self', 'taxonomy', '_label', 'self', 'taxonomy', 'label', 'KeywordLabel', '_label', 'intersection', 'list', 'label', 'keywords', 'intersection', 'Multiset', 'content', 'split', 'intersection', 'Counter', 'intersection', 'node_labels', 'label', 'index', 'sum', 'intersection', 'k', 'label', 'weights', 'k', 'k', 'intersection', 'keys', 'norm', 'np', 'sum', 'node_labels', 'node_vec', 'node_labels', 'norm', 'norm', 'np', 'zeros', 'len', 'self', 'taxonomy', 'node_vec']


In [19]:
gt = ['collections', 'Counter', 'numpy', 'np', 'multiset', 'Multiset', 'entity',
      'taxonomy', 'KeywordLabel', 'annotation', 'LFBase', 'KeywordLF', 'LFBase',
      'annotate', 'self', 'name', 'str', 'content', 'str', 'np', 'array',
      'node_labels', 'np', 'zeros', 'len', 'self', 'taxonomy', '_label', 'self', 'taxonomy',
      'label', 'KeywordLabel', '_label', 'intersection', 'list', 'label', 'keywords', 'intersection',
      'Multiset', 'content', 'split', 'intersection', 'Counter', 'intersection', 'node_labels', 'label',
      'index', 'sum', 'intersection', 'k', 'label', 'weights', 'k', 'k', 'intersection', 'keys',
      'norm', 'np', 'sum', 'node_labels', 'node_vec', 'node_labels', 'norm', 'norm', '0', 'np', 'zeros',
      'len', 'self', 'taxonomy', 'node_vec']

In [20]:
len(gt)

71

In [22]:
from multiset import Multiset

Multiset(res) - Multiset(gt)

Multiset({})